In [7]:
# import packages
import numpy as np
import pandas as pd

from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer

#lime
from lime.lime_text import LimeTextExplainer
from collections import defaultdict
from tqdm import tqdm

In [8]:
# load sample data
all_train = pd.read_csv('fold_labels/train_all.csv', header = 0)[["phrase_stem_emoticon_generic", "phrase", "label"]]
all_test = pd.read_csv('fold_labels/test.csv', header = 0)[["phrase_stem_emoticon_generic", "phrase", "label"]]
all_sample = pd.concat([all_train, all_test], axis=0).reset_index().drop('index', axis=1)

In [9]:
all_sample.head()

,phrase_stem_emoticon_generic,phrase,label
0,grill fish good,grilled fish good,1.0
1,good,good,1.0
2,prawn chee cheong fun meh foolish thought cust...,prawn chee cheong fun meh foolish thought cust...,0.0
3,instead went pig conge lo mai kai crystal dump...,instead went pig congee lo mai kai crystal dum...,1.0
4,fun fact friendli owner,fun fact friendly owner,1.0


# LIME

In [12]:
def append_dict_of_scores(d, lime_exp, label):
    l = lime_exp.as_list(label=label)
    for item in l:
        key = item[0]
        val = item[1]
        if key in d:
            d[key].append(val)
        else:
            d[key] = [val]


def generate_lime_scores(pipe):
    d_neg = defaultdict()
    d_neu = defaultdict()
    d_pos = defaultdict()
    class_names = [-1, 0, 1]
    explainer = LimeTextExplainer(class_names = class_names, random_state=42)
    for i in tqdm(range(len(all_sample))):
        current_text = all_sample["phrase_stem_emoticon_generic"].iloc[i]
        exp = explainer.explain_instance(current_text, pipe.predict_proba, labels=[-1,0,1]) #1,-1
        append_dict_of_scores(d_neg, exp, -1)
        append_dict_of_scores(d_neu, exp, 0)
        append_dict_of_scores(d_pos, exp, 1)
    return d_neg, d_neu, d_pos

In [13]:
def dict_to_df(d, newcols):
    token_df =  pd.DataFrame([d]).T
    token_df = token_df.reset_index()
    token_df.columns = newcols
    return token_df

In [14]:
def get_token_avg_score(dpos, dneu, dneg):
    avgDict_pos = {}
    avgDict_neu = {}
    avgDict_neg = {}
    for k,v in dpos.items():
        # v is the list of impact on probability of predicting a class for a particular token
        avgDict_pos[k] = sum(v)/ float(len(v))
    for k,v in dneg.items():
        # v is the list of impact on probability of predicting a class for a particular token
        avgDict_neg[k] = sum(v)/ float(len(v))
    for k,v in dneu.items():
        # v is the list of impact on probability of predicting a class for a particular token
        avgDict_neu[k] = sum(v)/ float(len(v))
    pos=dict_to_df(avgDict_pos, ['token', 'average_pos_impact'])
    neg=dict_to_df(avgDict_neg, ['token', 'average_neg_impact'])
    neu=dict_to_df(avgDict_neu, ['token', 'average_neu_impact'])
    fin = pos.merge(neg, on='token', how = 'inner').merge(neu, on='token', how = 'inner')
    fin.sort_values(['average_pos_impact'], ascending=False)
    return fin

# NAIVES BAYES

In [15]:
# create pipeline
vec = CountVectorizer(analyzer="word",
    lowercase= True,
    ngram_range =(1,1),
    max_df = 0.25,
    min_df = 10)
nb = MultinomialNB(alpha = 0.5)
pipe_nb = make_pipeline(vec, nb)
pipe_nb.fit(all_train.phrase_stem_emoticon_generic, all_train.label)

Pipeline(memory=None,
         steps=[('countvectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=0.25,
                                 max_features=None, min_df=10,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('multinomialnb',
                 MultinomialNB(alpha=0.5, class_prior=None, fit_prior=True))],
         verbose=False)

# LIME 

In [17]:
dneg, dneu, dpos = generate_lime_scores(pipe_nb)

  0%|                                                                                         | 0/3261 [00:00<?, ?it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  0%|                                                                                 | 1/3261 [00:00<22:16,  2.44it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  0%|                                                                                 | 2/3261 [00:00<21:23,  2.54it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  0%|                                                                                 | 

  1%|▋                                                                               | 27/3261 [00:15<29:07,  1.85it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  1%|▋                                                                               | 28/3261 [00:16<29:55,  1.80it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  1%|▋                                                                               | 29/3261 [00:16<28:35,  1.88it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  1%|▋                                                                               | 3

  2%|█▎                                                                              | 54/3261 [00:29<26:45,  2.00it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  2%|█▎                                                                              | 55/3261 [00:29<27:25,  1.95it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  2%|█▎                                                                              | 56/3261 [00:30<26:39,  2.00it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  2%|█▍                                                                              | 5

  2%|█▉                                                                              | 81/3261 [00:44<28:00,  1.89it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  3%|██                                                                              | 82/3261 [00:44<27:52,  1.90it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  3%|██                                                                              | 83/3261 [00:44<25:34,  2.07it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  3%|██                                                                              | 8

  3%|██▌                                                                            | 108/3261 [00:59<28:28,  1.85it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  3%|██▋                                                                            | 109/3261 [00:59<29:55,  1.76it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  3%|██▋                                                                            | 110/3261 [01:00<27:56,  1.88it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  3%|██▋                                                                            | 11

...

 96%|██████████████████████████████████████████████████████████████████████████▉   | 3132/3261 [29:43<01:09,  1.86it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 96%|██████████████████████████████████████████████████████████████████████████▉   | 3133/3261 [29:44<01:01,  2.07it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 96%|██████████████████████████████████████████████████████████████████████████▉   | 3134/3261 [29:44<01:01,  2.08it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 96%|██████████████████████████████████████████████████████████████████████████▉   | 313

 97%|███████████████████████████████████████████████████████████████████████████▌  | 3159/3261 [29:57<00:48,  2.12it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 97%|███████████████████████████████████████████████████████████████████████████▌  | 3160/3261 [29:57<00:44,  2.30it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 97%|███████████████████████████████████████████████████████████████████████████▌  | 3161/3261 [29:58<00:43,  2.31it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 97%|███████████████████████████████████████████████████████████████████████████▋  | 316

 98%|████████████████████████████████████████████████████████████████████████████▏ | 3186/3261 [30:10<00:37,  2.01it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 98%|████████████████████████████████████████████████████████████████████████████▏ | 3187/3261 [30:10<00:35,  2.08it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 98%|████████████████████████████████████████████████████████████████████████████▎ | 3188/3261 [30:11<00:32,  2.23it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 98%|████████████████████████████████████████████████████████████████████████████▎ | 318

 99%|████████████████████████████████████████████████████████████████████████████▊ | 3213/3261 [30:24<00:25,  1.90it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 99%|████████████████████████████████████████████████████████████████████████████▉ | 3214/3261 [30:24<00:23,  2.01it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 99%|████████████████████████████████████████████████████████████████████████████▉ | 3215/3261 [30:25<00:25,  1.80it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 99%|████████████████████████████████████████████████████████████████████████████▉ | 321

 99%|█████████████████████████████████████████████████████████████████████████████▍| 3240/3261 [30:37<00:09,  2.17it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 99%|█████████████████████████████████████████████████████████████████████████████▌| 3241/3261 [30:37<00:10,  1.86it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 99%|█████████████████████████████████████████████████████████████████████████████▌| 3242/3261 [30:38<00:10,  1.87it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 99%|█████████████████████████████████████████████████████████████████████████████▌| 324

In [18]:
nb_lime = get_token_avg_score(dpos, dneu, dneg)
nb_lime.to_csv('explain_results/nb_lime.csv', index=False)

# RF

In [19]:
# create pipeline
vec_rf = CountVectorizer(analyzer="word",
    lowercase= True,
    ngram_range =(1,1),
    max_df = 1.0,
    min_df = 1)
rf = RandomForestClassifier(criterion = "gini", min_samples_split = 5, class_weight=None, max_features="auto", min_samples_leaf=1)
pipe_rf = make_pipeline(vec_rf, rf)
pipe_rf.fit(all_train.phrase_stem_emoticon_generic, all_train.label)

C:\Users\65918\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('countvectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, voc...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0

# LIME

In [20]:
dpos, dneu, dneg = generate_lime_scores(pipe_rf)

  0%|                                                                                         | 0/3261 [00:00<?, ?it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  0%|                                                                                 | 1/3261 [00:00<29:39,  1.83it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  0%|                                                                                 | 2/3261 [00:01<27:00,  2.01it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  0%|                                                                                 | 

  1%|▋                                                                               | 27/3261 [00:14<30:29,  1.77it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  1%|▋                                                                               | 28/3261 [00:15<29:18,  1.84it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  1%|▋                                                                               | 29/3261 [00:15<28:55,  1.86it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  1%|▋                                                                               | 3

  2%|█▎                                                                              | 54/3261 [00:28<27:43,  1.93it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  2%|█▎                                                                              | 55/3261 [00:29<29:27,  1.81it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  2%|█▎                                                                              | 56/3261 [00:30<27:59,  1.91it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  2%|█▍                                                                              | 5

  2%|█▉                                                                              | 81/3261 [00:43<27:13,  1.95it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  3%|██                                                                              | 82/3261 [00:44<28:02,  1.89it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  3%|██                                                                              | 83/3261 [00:44<29:00,  1.83it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  3%|██                                                                              | 8

  3%|██▌                                                                            | 108/3261 [00:58<30:55,  1.70it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  3%|██▋                                                                            | 109/3261 [00:59<31:12,  1.68it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  3%|██▋                                                                            | 110/3261 [00:59<29:06,  1.80it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  3%|██▋                                                                            | 11

...

 96%|██████████████████████████████████████████████████████████████████████████▉   | 3132/3261 [30:00<01:25,  1.50it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 96%|██████████████████████████████████████████████████████████████████████████▉   | 3133/3261 [30:00<01:22,  1.54it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 96%|██████████████████████████████████████████████████████████████████████████▉   | 3134/3261 [30:01<01:18,  1.62it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 96%|██████████████████████████████████████████████████████████████████████████▉   | 313

 97%|███████████████████████████████████████████████████████████████████████████▌  | 3159/3261 [30:15<00:51,  1.98it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 97%|███████████████████████████████████████████████████████████████████████████▌  | 3160/3261 [30:15<00:49,  2.05it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 97%|███████████████████████████████████████████████████████████████████████████▌  | 3161/3261 [30:16<00:50,  1.97it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 97%|███████████████████████████████████████████████████████████████████████████▋  | 316

 98%|████████████████████████████████████████████████████████████████████████████▏ | 3186/3261 [30:30<00:48,  1.56it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 98%|████████████████████████████████████████████████████████████████████████████▏ | 3187/3261 [30:31<00:47,  1.55it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 98%|████████████████████████████████████████████████████████████████████████████▎ | 3188/3261 [30:32<00:45,  1.59it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 98%|████████████████████████████████████████████████████████████████████████████▎ | 318

 99%|████████████████████████████████████████████████████████████████████████████▊ | 3213/3261 [30:49<00:30,  1.56it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 99%|████████████████████████████████████████████████████████████████████████████▉ | 3214/3261 [30:50<00:32,  1.45it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 99%|████████████████████████████████████████████████████████████████████████████▉ | 3215/3261 [30:51<00:32,  1.41it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 99%|████████████████████████████████████████████████████████████████████████████▉ | 321

 99%|█████████████████████████████████████████████████████████████████████████████▍| 3240/3261 [31:07<00:12,  1.62it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 99%|█████████████████████████████████████████████████████████████████████████████▌| 3241/3261 [31:08<00:13,  1.46it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 99%|█████████████████████████████████████████████████████████████████████████████▌| 3242/3261 [31:08<00:11,  1.62it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 99%|█████████████████████████████████████████████████████████████████████████████▌| 324

In [21]:
rf_lime = get_token_avg_score(dpos, dneu, dneg)
rf_lime.to_csv('explain_results/rf_lime.csv', index=False)

# SVM

In [22]:
# create pipeline
vec_svm = CountVectorizer(analyzer="word",
    lowercase= True,
    ngram_range =(1,1),
    max_df = 0.25,
    min_df = 1)
svm = SVC(C=5, kernel='rbf', probability=True,class_weight=None,gamma='scale')
pipe_svm = make_pipeline(vec_svm, svm)
pipe_svm.fit(all_train.phrase, all_train.label)

Pipeline(memory=None,
         steps=[('countvectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=0.25,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('svc',
                 SVC(C=5, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='scale',
                     kernel='rbf', max_iter=-1, probability=True,
                     random_state=None, shrinking=True, tol=0.001,
                 

# LIME

In [23]:
dpos, dneu, dneg = generate_lime_scores(pipe_svm)

  0%|                                                                                         | 0/3261 [00:00<?, ?it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  0%|                                                                               | 1/3261 [00:01<1:08:13,  1.26s/it]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  0%|                                                                               | 2/3261 [00:02<1:03:44,  1.17s/it]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  0%|                                                                               | 3/

  1%|▋                                                                             | 27/3261 [00:39<1:13:25,  1.36s/it]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  1%|▋                                                                             | 28/3261 [00:40<1:15:57,  1.41s/it]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  1%|▋                                                                             | 29/3261 [00:42<1:21:58,  1.52s/it]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  1%|▋                                                                             | 30/

  2%|█▎                                                                            | 54/3261 [01:13<1:07:04,  1.26s/it]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  2%|█▎                                                                            | 55/3261 [01:15<1:18:02,  1.46s/it]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  2%|█▎                                                                            | 56/3261 [01:16<1:09:31,  1.30s/it]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  2%|█▎                                                                            | 57/

  2%|█▉                                                                            | 81/3261 [01:48<1:00:57,  1.15s/it]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  3%|█▉                                                                            | 82/3261 [01:50<1:06:57,  1.26s/it]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  3%|█▉                                                                            | 83/3261 [01:51<1:03:47,  1.20s/it]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  3%|██                                                                            | 84/

  3%|██▌                                                                          | 108/3261 [02:24<1:14:41,  1.42s/it]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  3%|██▌                                                                          | 109/3261 [02:26<1:20:23,  1.53s/it]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  3%|██▌                                                                          | 110/3261 [02:27<1:13:02,  1.39s/it]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
  3%|██▌                                                                          | 111/

...

 96%|████████████████████████████████████████████████████████████████████████▉   | 3132/3261 [1:09:38<01:51,  1.15it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 96%|█████████████████████████████████████████████████████████████████████████   | 3133/3261 [1:09:38<01:40,  1.28it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 96%|█████████████████████████████████████████████████████████████████████████   | 3134/3261 [1:09:39<01:37,  1.30it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 96%|█████████████████████████████████████████████████████████████████████████   | 3135/

 97%|█████████████████████████████████████████████████████████████████████████▌  | 3159/3261 [1:10:02<01:38,  1.04it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 97%|█████████████████████████████████████████████████████████████████████████▋  | 3160/3261 [1:10:03<01:27,  1.15it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 97%|█████████████████████████████████████████████████████████████████████████▋  | 3161/3261 [1:10:03<01:27,  1.14it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 97%|█████████████████████████████████████████████████████████████████████████▋  | 3162/

 98%|██████████████████████████████████████████████████████████████████████████▎ | 3186/3261 [1:10:22<00:55,  1.36it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 98%|██████████████████████████████████████████████████████████████████████████▎ | 3187/3261 [1:10:23<00:51,  1.44it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 98%|██████████████████████████████████████████████████████████████████████████▎ | 3188/3261 [1:10:23<00:50,  1.44it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 98%|██████████████████████████████████████████████████████████████████████████▎ | 3189/

 99%|██████████████████████████████████████████████████████████████████████████▉ | 3213/3261 [1:10:46<00:41,  1.14it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 99%|██████████████████████████████████████████████████████████████████████████▉ | 3214/3261 [1:10:47<00:44,  1.05it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 99%|██████████████████████████████████████████████████████████████████████████▉ | 3215/3261 [1:10:48<00:51,  1.11s/it]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 99%|██████████████████████████████████████████████████████████████████████████▉ | 3216/

 99%|███████████████████████████████████████████████████████████████████████████▌| 3240/3261 [1:11:13<00:20,  1.04it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 99%|███████████████████████████████████████████████████████████████████████████▌| 3241/3261 [1:11:14<00:21,  1.10s/it]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 99%|███████████████████████████████████████████████████████████████████████████▌| 3242/3261 [1:11:15<00:18,  1.05it/s]C:\Users\65918\Anaconda3\lib\site-packages\lime\lime_text.py:114: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
 99%|███████████████████████████████████████████████████████████████████████████▌| 3243/

In [24]:
svm_lime = get_token_avg_score(dpos, dneu, dneg)
svm_lime.to_csv('explain_results/svm_lime.csv', index=False)